In [35]:
import os
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import ast

In [36]:
pd.options.display.max_columns = None

In [37]:
load_dotenv()

True

df = pd.read_csv('./api_keys.csv')
zwsid = df.loc[df['API'] == 'zillow']['KEY'].iloc[0]
rapid_api_key = df.loc[df['API'] == 'rapid']['KEY'].iloc[0]

# API Call using "Rapid API" developed by market place vendor

In [38]:
rapid = os.getenv("RAPID")
zillow = os.getenv("ZILLOW")

In [39]:
load_dotenv()
import requests

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring = {"location": "austin, tx", "status_type":"ForRent", "price":""}

headers = {
    "X-RapidAPI-Key": rapid,
    "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}
z_rent = requests.request("GET", url, headers=headers, params=querystring)

# Understand query results. Reveals data structure, number of rows per page, and number of pages. These are settings from the API

In [40]:
z_rent

<Response [200]>

In [41]:
display(z_rent)

<Response [200]>

In [42]:
z_rent_json = z_rent.json()

In [44]:
z_rent_json

{'props': [{'units': [{'beds': '0', 'price': '$1,743+'},
    {'beds': '1', 'price': '$2,039+'},
    {'beds': '2', 'price': '$2,995+'}],
   'listingStatus': 'FOR_RENT',
   'zpid': '30.268095--97.75174',
   'longitude': -97.75174,
   'buildingName': 'Gables West Avenue',
   'address': '300 West Ave, Austin, TX',
   'isBuilding': True,
   'hasImage': True,
   'lotId': 1001413894,
   'imgSrc': 'https://photos.zillowstatic.com/fp/acac58c015293e2ee47c0296ab0f74ee-p_e.jpg',
   'latitude': 30.268095},
  {'units': [{'beds': '1', 'price': '$2,164+'},
    {'beds': '2', 'price': '$3,943+'}],
   'listingStatus': 'FOR_RENT',
   'zpid': '30.271212--97.73154',
   'longitude': -97.73154,
   'buildingName': 'Avenir',
   'address': '1109 N Interstate Highway 35, Austin, TX',
   'isBuilding': True,
   'hasImage': True,
   'lotId': 2196353113,
   'imgSrc': 'https://photos.zillowstatic.com/fp/b28e13f4c5a1706556412ded22486a00-p_e.jpg',
   'latitude': 30.271212},
  {'units': [{'beds': '1', 'price': '$1,427+'}

In [45]:
z_rent_json.keys()

dict_keys(['props', 'resultsPerPage', 'totalResultCount', 'totalPages'])

In [46]:
total_pages = z_rent_json['totalPages']
total_pages

20

In [47]:
total_pages_list = list(range(1,total_pages + 1))
total_pages_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [48]:
columns=['units', 'price', 'bedrooms', 'propertyType','buildingName', 'zpid', 'longitude','latitude','address']

## Create "for loop" to capture all pages available from the API call. Use the "df.append" function to iterate results after every loop.

In [49]:
z_rent_json.keys()

dict_keys(['props', 'resultsPerPage', 'totalResultCount', 'totalPages'])

In [50]:
z_rent_json = pd.json_normalize(data= z_rent_json['props'])

In [51]:
data_df=pd.DataFrame(columns=columns)

for page in total_pages_list[:20]:
    querystring = {"location": "seattle, wa", "page":page , "status_type":"ForRent", "price":""}
    z_rent_json = requests.request("GET", url, headers=headers, params=querystring)
    z_rent_json = z_rent.json()
    z_rent_json = pd.json_normalize(z_rent_json['props']).iloc[:,:9]
    data_df = data_df.append(z_rent_json,ignore_index=True)

In [52]:
time.sleep(1.0)

In [53]:
data_df

,units,price,bedrooms,propertyType,buildingName,zpid,longitude,latitude,address,listingStatus,isBuilding,hasImage,lotId
0,"[{'beds': '0', 'price': '$1,743+'}, {'beds': '...",NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.751740,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09
1,"[{'beds': '1', 'price': '$2,164+'}, {'beds': '...",NaN,NaN,NaN,Avenir,30.271212--97.73154,-97.731540,NaN,"1109 N Interstate Highway 35, Austin, TX",FOR_RENT,True,True,2.196353e+09
2,"[{'beds': '1', 'price': '$1,427+'}, {'beds': '...",NaN,NaN,NaN,Farmhouse Apartments,30.144293--97.79303,-97.793030,NaN,"10801 S Interstate Highway 35 Frontage Rd, Aus...",FOR_RENT,True,True,2.402567e+09
3,"[{'beds': '1', 'price': '$1,564+'}, {'beds': '...",NaN,NaN,NaN,Lantana Hills,30.253077--97.872826,-97.872826,NaN,"7601 Rialto Blvd, Austin, TX",FOR_RENT,True,True,1.001414e+09
4,"[{'beds': '1', 'price': '$1,302+'}, {'beds': '...",NaN,NaN,NaN,Terrazzo,30.43188--97.77782,-97.777820,NaN,"8585 Spicewood Springs Rd, Austin, TX",FOR_RENT,True,True,1.001414e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,"[{'beds': '0', 'price': '$1,463+'}, {'beds': '...",NaN,NaN,NaN,3Waller,30.263382--97.736305,-97.736305,NaN,"710 E 3rd St, Austin, TX",FOR_RENT,True,True,2.372307e+09
796,NaN,NaN,NaN,NaN,NaN,2071212849,-97.717360,NaN,"1020 Ellingson Ln #A, Austin, TX 78751",FOR_RENT,NaN,True,NaN
797,"[{'beds': '1', 'price': '$1,875+'}, {'beds': '...",NaN,NaN,NaN,The Tuscany Apartments,30.28121--97.758026,-97.758026,NaN,"1301 W Lynn St, Austin, TX",FOR_RENT,True,True,1.001414e+09
798,"[{'beds': '0', 'price': '$1,289+'}, {'beds': '...",NaN,NaN,NaN,Bexley at Tech Ridge,30.396175--97.65264,-97.652640,NaN,"1200 E Parmer Ln, Austin, TX",FOR_RENT,True,True,1.001642e+09


In [54]:
data_df_expanded = data_df.explode('units').reset_index()

In [55]:
data_df_expanded

,index,units,price,bedrooms,propertyType,buildingName,zpid,longitude,latitude,address,listingStatus,isBuilding,hasImage,lotId
0,0,"{'beds': '0', 'price': '$1,743+'}",NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09
1,0,"{'beds': '1', 'price': '$2,039+'}",NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09
2,0,"{'beds': '2', 'price': '$2,995+'}",NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09
3,1,"{'beds': '1', 'price': '$2,164+'}",NaN,NaN,NaN,Avenir,30.271212--97.73154,-97.73154,NaN,"1109 N Interstate Highway 35, Austin, TX",FOR_RENT,True,True,2.196353e+09
4,1,"{'beds': '2', 'price': '$3,943+'}",NaN,NaN,NaN,Avenir,30.271212--97.73154,-97.73154,NaN,"1109 N Interstate Highway 35, Austin, TX",FOR_RENT,True,True,2.196353e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,798,"{'beds': '1', 'price': '$1,424+'}",NaN,NaN,NaN,Bexley at Tech Ridge,30.396175--97.65264,-97.65264,NaN,"1200 E Parmer Ln, Austin, TX",FOR_RENT,True,True,1.001642e+09
1936,798,"{'beds': '2', 'price': '$1,714+'}",NaN,NaN,NaN,Bexley at Tech Ridge,30.396175--97.65264,-97.65264,NaN,"1200 E Parmer Ln, Austin, TX",FOR_RENT,True,True,1.001642e+09
1937,799,"{'beds': '1', 'price': '$1,200+'}",NaN,NaN,NaN,Avana SoCo,30.20007--97.77253,-97.77253,NaN,"401 Little Texas Ln, Austin, TX",FOR_RENT,True,True,1.001414e+09
1938,799,"{'beds': '2', 'price': '$1,598+'}",NaN,NaN,NaN,Avana SoCo,30.20007--97.77253,-97.77253,NaN,"401 Little Texas Ln, Austin, TX",FOR_RENT,True,True,1.001414e+09


# Trimming the dataframe. Removing columns that will not be needed

In [56]:
data_df_expanded = data_df_expanded.drop(['index'], axis=1)
data_df_expanded

,units,price,bedrooms,propertyType,buildingName,zpid,longitude,latitude,address,listingStatus,isBuilding,hasImage,lotId
0,"{'beds': '0', 'price': '$1,743+'}",NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09
1,"{'beds': '1', 'price': '$2,039+'}",NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09
2,"{'beds': '2', 'price': '$2,995+'}",NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09
3,"{'beds': '1', 'price': '$2,164+'}",NaN,NaN,NaN,Avenir,30.271212--97.73154,-97.73154,NaN,"1109 N Interstate Highway 35, Austin, TX",FOR_RENT,True,True,2.196353e+09
4,"{'beds': '2', 'price': '$3,943+'}",NaN,NaN,NaN,Avenir,30.271212--97.73154,-97.73154,NaN,"1109 N Interstate Highway 35, Austin, TX",FOR_RENT,True,True,2.196353e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,"{'beds': '1', 'price': '$1,424+'}",NaN,NaN,NaN,Bexley at Tech Ridge,30.396175--97.65264,-97.65264,NaN,"1200 E Parmer Ln, Austin, TX",FOR_RENT,True,True,1.001642e+09
1936,"{'beds': '2', 'price': '$1,714+'}",NaN,NaN,NaN,Bexley at Tech Ridge,30.396175--97.65264,-97.65264,NaN,"1200 E Parmer Ln, Austin, TX",FOR_RENT,True,True,1.001642e+09
1937,"{'beds': '1', 'price': '$1,200+'}",NaN,NaN,NaN,Avana SoCo,30.20007--97.77253,-97.77253,NaN,"401 Little Texas Ln, Austin, TX",FOR_RENT,True,True,1.001414e+09
1938,"{'beds': '2', 'price': '$1,598+'}",NaN,NaN,NaN,Avana SoCo,30.20007--97.77253,-97.77253,NaN,"401 Little Texas Ln, Austin, TX",FOR_RENT,True,True,1.001414e+09


In [23]:
#data_df_expanded = data_df_expanded.drop(['lotId','hasImage','isBuilding','listingStatus'], axis=1)
#data_df_expanded

In [57]:
data_df_expanded

,units,price,bedrooms,propertyType,buildingName,zpid,longitude,latitude,address,listingStatus,isBuilding,hasImage,lotId
0,"{'beds': '0', 'price': '$1,743+'}",NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09
1,"{'beds': '1', 'price': '$2,039+'}",NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09
2,"{'beds': '2', 'price': '$2,995+'}",NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09
3,"{'beds': '1', 'price': '$2,164+'}",NaN,NaN,NaN,Avenir,30.271212--97.73154,-97.73154,NaN,"1109 N Interstate Highway 35, Austin, TX",FOR_RENT,True,True,2.196353e+09
4,"{'beds': '2', 'price': '$3,943+'}",NaN,NaN,NaN,Avenir,30.271212--97.73154,-97.73154,NaN,"1109 N Interstate Highway 35, Austin, TX",FOR_RENT,True,True,2.196353e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,"{'beds': '1', 'price': '$1,424+'}",NaN,NaN,NaN,Bexley at Tech Ridge,30.396175--97.65264,-97.65264,NaN,"1200 E Parmer Ln, Austin, TX",FOR_RENT,True,True,1.001642e+09
1936,"{'beds': '2', 'price': '$1,714+'}",NaN,NaN,NaN,Bexley at Tech Ridge,30.396175--97.65264,-97.65264,NaN,"1200 E Parmer Ln, Austin, TX",FOR_RENT,True,True,1.001642e+09
1937,"{'beds': '1', 'price': '$1,200+'}",NaN,NaN,NaN,Avana SoCo,30.20007--97.77253,-97.77253,NaN,"401 Little Texas Ln, Austin, TX",FOR_RENT,True,True,1.001414e+09
1938,"{'beds': '2', 'price': '$1,598+'}",NaN,NaN,NaN,Avana SoCo,30.20007--97.77253,-97.77253,NaN,"401 Little Texas Ln, Austin, TX",FOR_RENT,True,True,1.001414e+09


In [58]:
import ast

In [59]:
counts = 10
count = 0
for row in data_df_expanded['units']:
    if not isinstance(row, dict):
        print(row, type(row))
        count+=1
        if count == counts:
            break

nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>


In [62]:
def data_clean(row):
    if isinstance(row, dict):
        return row
    if isinstance(row, str):
        return ast.literal_eval(row)
    else:
        return pd.NA

In [64]:
data_df_expanded['units'].replace(pd.NA,'{}', inplace=True)
data_df_expanded['units_dict'] = data_df_expanded['units'].apply(data_clean)
data_df_expanded['units_dict']

0       {'beds': '0', 'price': '$1,743+'}
1       {'beds': '1', 'price': '$2,039+'}
2       {'beds': '2', 'price': '$2,995+'}
3       {'beds': '1', 'price': '$2,164+'}
4       {'beds': '2', 'price': '$3,943+'}
                      ...                
1935    {'beds': '1', 'price': '$1,424+'}
1936    {'beds': '2', 'price': '$1,714+'}
1937    {'beds': '1', 'price': '$1,200+'}
1938    {'beds': '2', 'price': '$1,598+'}
1939    {'beds': '3', 'price': '$2,525+'}
Name: units_dict, Length: 1940, dtype: object

In [65]:
data_as_columns = pd.json_normalize(data_df_expanded['units_dict'])
data_as_columns.columns = ["units.beds", "units.price"]
data_as_columns

,units.beds,units.price
0,0,"$1,743+"
1,1,"$2,039+"
2,2,"$2,995+"
3,1,"$2,164+"
4,2,"$3,943+"
...,...,...
1935,1,"$1,424+"
1936,2,"$1,714+"
1937,1,"$1,200+"
1938,2,"$1,598+"


In [66]:
joined_data = pd.concat([data_df_expanded, data_as_columns], axis=1)
joined_data

,units,price,bedrooms,propertyType,buildingName,zpid,longitude,latitude,address,listingStatus,isBuilding,hasImage,lotId,units_dict,units.beds,units.price
0,"{'beds': '0', 'price': '$1,743+'}",NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09,"{'beds': '0', 'price': '$1,743+'}",0,"$1,743+"
1,"{'beds': '1', 'price': '$2,039+'}",NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09,"{'beds': '1', 'price': '$2,039+'}",1,"$2,039+"
2,"{'beds': '2', 'price': '$2,995+'}",NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09,"{'beds': '2', 'price': '$2,995+'}",2,"$2,995+"
3,"{'beds': '1', 'price': '$2,164+'}",NaN,NaN,NaN,Avenir,30.271212--97.73154,-97.73154,NaN,"1109 N Interstate Highway 35, Austin, TX",FOR_RENT,True,True,2.196353e+09,"{'beds': '1', 'price': '$2,164+'}",1,"$2,164+"
4,"{'beds': '2', 'price': '$3,943+'}",NaN,NaN,NaN,Avenir,30.271212--97.73154,-97.73154,NaN,"1109 N Interstate Highway 35, Austin, TX",FOR_RENT,True,True,2.196353e+09,"{'beds': '2', 'price': '$3,943+'}",2,"$3,943+"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,"{'beds': '1', 'price': '$1,424+'}",NaN,NaN,NaN,Bexley at Tech Ridge,30.396175--97.65264,-97.65264,NaN,"1200 E Parmer Ln, Austin, TX",FOR_RENT,True,True,1.001642e+09,"{'beds': '1', 'price': '$1,424+'}",1,"$1,424+"
1936,"{'beds': '2', 'price': '$1,714+'}",NaN,NaN,NaN,Bexley at Tech Ridge,30.396175--97.65264,-97.65264,NaN,"1200 E Parmer Ln, Austin, TX",FOR_RENT,True,True,1.001642e+09,"{'beds': '2', 'price': '$1,714+'}",2,"$1,714+"
1937,"{'beds': '1', 'price': '$1,200+'}",NaN,NaN,NaN,Avana SoCo,30.20007--97.77253,-97.77253,NaN,"401 Little Texas Ln, Austin, TX",FOR_RENT,True,True,1.001414e+09,"{'beds': '1', 'price': '$1,200+'}",1,"$1,200+"
1938,"{'beds': '2', 'price': '$1,598+'}",NaN,NaN,NaN,Avana SoCo,30.20007--97.77253,-97.77253,NaN,"401 Little Texas Ln, Austin, TX",FOR_RENT,True,True,1.001414e+09,"{'beds': '2', 'price': '$1,598+'}",2,"$1,598+"


In [71]:
final_df = joined_data.drop(columns=['units', 'units_dict'])
final_df

,price,bedrooms,propertyType,buildingName,zpid,longitude,latitude,address,listingStatus,isBuilding,hasImage,lotId,units.beds,units.price
0,NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09,0,"$1,743+"
1,NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09,1,"$2,039+"
2,NaN,NaN,NaN,Gables West Avenue,30.268095--97.75174,-97.75174,NaN,"300 West Ave, Austin, TX",FOR_RENT,True,True,1.001414e+09,2,"$2,995+"
3,NaN,NaN,NaN,Avenir,30.271212--97.73154,-97.73154,NaN,"1109 N Interstate Highway 35, Austin, TX",FOR_RENT,True,True,2.196353e+09,1,"$2,164+"
4,NaN,NaN,NaN,Avenir,30.271212--97.73154,-97.73154,NaN,"1109 N Interstate Highway 35, Austin, TX",FOR_RENT,True,True,2.196353e+09,2,"$3,943+"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,NaN,NaN,NaN,Bexley at Tech Ridge,30.396175--97.65264,-97.65264,NaN,"1200 E Parmer Ln, Austin, TX",FOR_RENT,True,True,1.001642e+09,1,"$1,424+"
1936,NaN,NaN,NaN,Bexley at Tech Ridge,30.396175--97.65264,-97.65264,NaN,"1200 E Parmer Ln, Austin, TX",FOR_RENT,True,True,1.001642e+09,2,"$1,714+"
1937,NaN,NaN,NaN,Avana SoCo,30.20007--97.77253,-97.77253,NaN,"401 Little Texas Ln, Austin, TX",FOR_RENT,True,True,1.001414e+09,1,"$1,200+"
1938,NaN,NaN,NaN,Avana SoCo,30.20007--97.77253,-97.77253,NaN,"401 Little Texas Ln, Austin, TX",FOR_RENT,True,True,1.001414e+09,2,"$1,598+"


# Saving file to CSV: Needed for MVP

In [72]:
final_df.to_csv("austin_tx.csv", index=False)

# Data Clean up

In [ ]:
final_df.keys()

In [ ]:
final_df